# Seattle non-residential buildings energy consumption

This notebook aims at predicting the total energy consumptions of non-residential buildings in the city of Seattle.
It relies on an official dataset available here: https://data.seattle.gov/dataset/2016-Building-Energy-Benchmarking/2bpz-gwpy
We are using a modified (by us) version of the database that you can find at this adress: inserer lien github

Another almost identical notebook exists and deals with GHG emissions rather than Energy consumption.

## Initialization

In [50]:
# Packages import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [51]:
# Loading the dataset
data = pd.read_csv(r'./Data/seattle_cleaned_dataset.csv', sep = ',', low_memory = False)

In [52]:
data.describe()

,OSEBuildingID,ZipCode,Latitude,Longitude,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ENERGYSTARScore,SiteEUIWN(kBtu/sf),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions,GHGEmissionsIntensity,BuildingAge
count,1636.000000,1620.000000,1636.000000,1636.000000,1636.000000,1636.000000,1.636000e+03,1636.000000,1.636000e+03,1078.000000,1636.000000,1.636000e+03,1.636000e+03,1.636000e+03,1.636000e+03,1636.000000,1636.000000,1636.000000
mean,16255.489609,98116.932716,47.616126,-122.332965,1.099633,4.118582,1.137481e+05,12945.718826,1.008023e+05,65.138219,75.018949,8.122033e+06,4.700248e+05,5.475530e+06,1.997737e+06,180.552152,1.626333,52.827628
std,13854.775408,18.533239,0.048417,0.024662,1.161766,6.565362,1.941333e+05,42535.544496,1.728464e+05,28.379155,74.910480,2.219025e+07,5.156614e+06,1.339365e+07,9.453094e+06,708.466267,2.352106,32.526775
min,1.000000,98006.000000,47.499170,-122.411820,0.000000,0.000000,1.128500e+04,0.000000,3.636000e+03,1.000000,0.000000,0.000000e+00,0.000000e+00,-1.154170e+05,0.000000e+00,-0.800000,-0.020000,0.000000
25%,577.750000,98105.000000,47.585160,-122.343335,1.000000,1.000000,2.950550e+04,0.000000,2.852325e+04,48.000000,36.099998,1.322253e+06,0.000000e+00,7.272688e+05,0.000000e+00,20.427500,0.360000,26.000000
50%,21131.000000,98110.000000,47.612380,-122.332890,1.000000,2.000000,4.971200e+04,0.000000,4.763750e+04,73.000000,54.299999,2.736046e+06,0.000000e+00,1.628064e+06,5.141630e+05,50.015000,0.880000,49.000000
75%,24591.500000,98125.000000,47.649760,-122.321765,1.000000,4.000000,1.060102e+05,0.000000,9.506750e+04,89.000000,85.299997,7.187220e+06,0.000000e+00,4.882877e+06,1.529470e+06,144.870000,1.910000,85.000000
max,50226.000000,98199.000000,47.733870,-122.258640,27.000000,99.000000,2.200000e+06,512608.000000,2.200000e+06,100.000000,834.400024,4.716139e+08,1.349435e+08,2.745325e+08,2.979090e+08,16870.980000,34.090000,115.000000


As minimums of Electricity(kBtu) and TotalGHGEmissions, we find negative values. As previously investigated, it corresponds to the Bullit Center, self-designated as "the Greenest Commercial Building in the World". Indeed, on average for a given year, it produces more energy that it requires to operate.

Considering it is not a measurement error but an outlier, we'll keep it for the moment, and maybe run two models: one with and one without this observation.

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   OSEBuildingID              1636 non-null   int64  
 1   PrimaryPropertyType        1636 non-null   object 
 2   PropertyName               1636 non-null   object 
 3   Address                    1636 non-null   object 
 4   ZipCode                    1620 non-null   float64
 5   Neighborhood               1636 non-null   object 
 6   Latitude                   1636 non-null   float64
 7   Longitude                  1636 non-null   float64
 8   NumberofBuildings          1636 non-null   float64
 9   NumberofFloors             1636 non-null   int64  
 10  PropertyGFATotal           1636 non-null   int64  
 11  PropertyGFAParking         1636 non-null   int64  
 12  PropertyGFABuilding(s)     1636 non-null   int64  
 13  ListOfAllPropertyUseTypes  1636 non-null   objec

All variables seems to be coded in the proper format.

We have some missing values for ZipCode, LargestPropertyUseType, and ENERGYSTARScore. We'll delete Zipcode (as well as other localization/id variables) but keep the two other.

## Preprocessing

In [54]:
# Exclude some 'id' and localization variables for now
data = data.drop(['OSEBuildingID', 'PropertyName', 'Address', 'ZipCode', 'ComplianceStatus'], axis = 1)

#### Numerical vs Categorical features decomposition

In [55]:
# Categorical features
categorical_features = data[['PrimaryPropertyType', 'Neighborhood', 
                             'ListOfAllPropertyUseTypes', 'LargestPropertyUseType']]
categorical_features.nunique()

PrimaryPropertyType           21
Neighborhood                  19
ListOfAllPropertyUseTypes    362
LargestPropertyUseType        55
dtype: int64

As seen in the data exploration notebook, there are a lot of PropertyUseTypes (362). On the other hand, PrimaryPropertyType (with 21 unique values) may be a little to innacurate.  
In between, LargestPropertyUseType has a interesting 55 unique values and will be retained as our 'building type' variable. So we have to delete the four observations that do not have a value for this feature.

In [56]:
data = data[~(data['LargestPropertyUseType'].isnull())]

In [57]:
categorical_features = data[['Neighborhood', 'LargestPropertyUseType']]

In [58]:
# Numerical features
numerical_features = data[['Latitude', 'Longitude',
                           'NumberofBuildings', 'NumberofFloors', 'BuildingAge',
                           'PropertyGFATotal','PropertyGFAParking', 'PropertyGFABuilding(s)',
                           'SiteEUIWN(kBtu/sf)', 'SiteEnergyUseWN(kBtu)',
                           'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)',
                           'TotalGHGEmissions', 'GHGEmissionsIntensity',
                           'ENERGYSTARScore']]

The retained numerical features for sure present multicolinearity. For example, the PropertyGFATotal is positively correlated with the NumberofBuildings.
Including both will not a problem with regards to the overall performance and prediction power of the model, but it will blur the explanation impact of each variable. We should keep that in mind.

ENERGYSTARScore will recieve a special treatment due to its relative low number of observations.

Our target is the energy consumption. Two variables measure that: 'SiteEUIWN(kBtu/sf)' and 'SiteEnergyUseWN(kBtu)'.
We will take the gross energy consumption: 'SiteEnergyUseWN(kBtu)' as feature to predict. The 'energy use intensity' ('SiteEUIWN(kBtu/sf)') will be deleted.

#### Train & Test sets

In [59]:
from sklearn.model_selection import train_test_split

X = data[['Latitude', 'Longitude', 
          'NumberofBuildings', 'NumberofFloors', 'BuildingAge',
          'PropertyGFATotal','PropertyGFAParking', 'PropertyGFABuilding(s)',
          'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)',
          'TotalGHGEmissions', 'GHGEmissionsIntensity',
          'Neighborhood', 'LargestPropertyUseType']]

y = data['SiteEnergyUseWN(kBtu)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## Models

We will include all remaining preprocessing in pipelines.

First, we are going to run two linear models: a LinearRegression (as baseline model) and an ElasticNet regression.
Then, we will implement two ensemble methods: a Random Forest and a XGBoost.

All R² and MSE will be stored in a table to compare results.  

In [60]:
# Selected Scikit-Learn modules
# Pipeline tools
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

# Transformers
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder

# Linear models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

### Baseline model

In [61]:
## Linear Regression
# Transformer
transformer = make_column_transformer(
                                      (RobustScaler(),  
                                       # We choose the robust scaler because we have outliers
                                       # that we want to keep
                                              ['Latitude', 'Longitude', 
                                               'NumberofBuildings', 'NumberofFloors', 'BuildingAge',
                                               'PropertyGFATotal','PropertyGFAParking', 'PropertyGFABuilding(s)',
                                               'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)',
                                               'TotalGHGEmissions', 'GHGEmissionsIntensity']),   
                                      (OneHotEncoder(handle_unknown='ignore'),
                                       # we decide to ignore unknown categories because some building types are
                                       # unique, and therefore only exist
                                       # in either the test or the train set.
                                              ['Neighborhood', 'LargestPropertyUseType'])
                                     )


model_lr = make_pipeline(transformer, LinearRegression())
model_lr.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('robustscaler',
                                                  RobustScaler(),
                                                  ['Latitude', 'Longitude',
                                                   'NumberofBuildings',
                                                   'NumberofFloors',
                                                   'BuildingAge',
                                                   'PropertyGFATotal',
                                                   'PropertyGFAParking',
                                                   'PropertyGFABuilding(s)',
                                                   'SteamUse(kBtu)',
                                                   'Electricity(kBtu)',
                                                   'NaturalGas(kBtu)',
                                                   'TotalGHGEmissions',
                                          

In [62]:
# Predictions
y_pred = model_lr.predict(X_test)

In [63]:
print("R² =", model_lr.score(X_train, y_train))
print("RMSE =", np.sqrt(mean_squared_error(y_test, y_pred)))

R² = 0.9947382847098875
RMSE = 645317.1962034381
